## Real-Time Visualisation

Jupyter data stream visualisation widgets can be created with a HTML snippet and the implementation of a few JavaScript functions. A python API python method is provided to inject these into the Jupyter interface and to connect them with the data steams of interest.

The three required functions are:

- `reset()` - called to initialise the visualisation
- `process()` - called for each data stream update
- `draw()` - called when the browser is ready to display a new image.

As data streams may be updated many times faster than the browser can display an image, the process function can be called many times more often than the draw.

A typical use of the visualisation API is to inject a HTML canvas into jupyter, and then update in real-time to display interesting state information from a running closed loop application.

The example below uses this [HTML File](assets/CL-04.%20Example%20Visualiser.html) and this [JavaScript File](assets/CL-04.%20Example%20Visualiser.mjs).

The HTML and JavaScript files are combined into a Jupyter widget by calling `display_visualiser()` along with the names of the data streams of interest:

In [1]:
from cl.visualisation.jupyter import display_visualiser

display_visualiser(
    html_file='assets/CL-04. Example Visualiser.html',
    javascript_file='assets/CL-04. Example Visualiser.mjs',
    data_streams=['gameplay']
    )

Side,Bounces
Left,
Right,
Top,
Bottom,
Corner,


This example subscribes to a custom 'gameplay' data stream, displays a canvas for drawing on, and adds a html table for attributes.

Lets create a simple moving ball simulation that publishes data to the 'gameplay' data stream (execute, and then scroll back up to look at the visualiser:

In [2]:
import cl
import time
import random
import math

DURATION_SECONDS   = 30
HZ                 = 100
TICK_DELTA_SECONDS = 1 / HZ

GAME_WIDTH         = 320
GAME_HEIGHT        = 180
BALL_WIDTH         = 10
BALL_HEIGHT        = 10

BALL_SPEED         = 1.5

HALF_BALL_WIDTH    = BALL_WIDTH  / 2
HALF_BALL_HEIGHT   = BALL_HEIGHT / 2

class Game:
    def __init__(self, data_stream):
        self.data_stream = data_stream

        self.x = random.randint(HALF_BALL_WIDTH,  GAME_WIDTH  - HALF_BALL_WIDTH)
        self.y = random.randint(HALF_BALL_HEIGHT, GAME_HEIGHT - HALF_BALL_HEIGHT)
    
        angle_radians = math.radians(random.randint(0, 359))
        self.vel_x = math.cos(angle_radians) * BALL_SPEED
        self.vel_y = math.sin(angle_radians) * BALL_SPEED

        self.bounces_left   = 0
        self.bounces_right  = 0
        self.bounces_top    = 0
        self.bounces_bottom = 0
        self.bounces_corner = 0

        data_stream.update_attributes(
            {
                'bounces_left':   self.bounces_left,
                'bounces_right':  self.bounces_right,
                'bounces_top':    self.bounces_top,
                'bounces_bottom': self.bounces_bottom,
                'bounces_corner': self.bounces_corner,
            })

    def tick(self):
        # Move
        self.x += self.vel_x
        self.y += self.vel_y

        bounce_count = 0

        # Process wall bounces
        if self.x < HALF_BALL_WIDTH:
            self.x += HALF_BALL_WIDTH - self.x
            self.vel_x = 0 - self.vel_x
            self.bounces_left += 1
            bounce_count += 1
            data_stream.set_attribute('bounces_left', self.bounces_left )
            
        elif self.x > GAME_WIDTH - HALF_BALL_WIDTH:
            self.x -= self.x - (GAME_WIDTH - HALF_BALL_WIDTH)
            self.vel_x = 0 - self.vel_x
            self.bounces_right += 1
            bounce_count += 1
            data_stream.set_attribute('bounces_right', self.bounces_right )
        
        if self.y < HALF_BALL_HEIGHT:
            self.y += HALF_BALL_HEIGHT - self.y
            self.vel_y = -self.vel_y
            self.bounces_top += 1
            bounce_count += 1
            data_stream.set_attribute('bounces_top', self.bounces_top )
        
        elif self.y > GAME_HEIGHT - HALF_BALL_HEIGHT:
            self.y -= self.y - (GAME_HEIGHT - HALF_BALL_HEIGHT)
            self.vel_y = -self.vel_y
            self.bounces_bottom += 1
            bounce_count += 1
            data_stream.set_attribute('bounces_bottom', self.bounces_bottom )

        if bounce_count == 2:
            self.bounces_corner += 1
            data_stream.set_attribute('bounces_corner', self.bounces_corner )

        self.data_stream.append(neurons.timestamp(), self.to_data_stream_update())

    def to_data_stream_update(self):
        return \
            {
                'x': self.x,
                'y': self.y
            }

with cl.open() as neurons:
    recording = neurons.record()
    
    data_stream = \
        neurons.create_data_stream(
            'gameplay',
            {
                'game_width':     GAME_WIDTH,
                'game_height':    GAME_HEIGHT,
                'ball_width':     BALL_WIDTH,
                'ball_height':    BALL_HEIGHT,
            })

    game = Game(data_stream)
    
    for tick in cl.ClosedLoop(
                    neurons,
                    HZ,
                    stop_after_seconds=DURATION_SECONDS
        ):
        game.tick()

    recording.stop()
    

Multiple independent visualisations can run within a single Jupyter notebook, and visualisations do not need to run within the same notebook as the application that is publishing the data stream.

## Next

[Reading Raw Data](CL-05.%20Reading%20Raw%20Data.ipynb)